### **4비트 양자화 QLoRA 파인튜닝: 개념과 코드 상세 설명**

---

### 1. **QLoRA란?**

QLoRA(Quantized Low-Rank Adaptation)는 **4비트 양자화**와 **LoRA(저랭크 어댑터)**를 활용한 효율적인 파인튜닝 기법입니다. 이 방식에서는 **모델의 모든 파라미터를 동결(Freeze)** 시키고, 가벼운 **추가 어댑터 파라미터만 학습**합니다. 이를 통해 매우 큰 언어 모델도 적은 메모리와 연산 자원으로 **효율적으로 파인튜닝**할 수 있습니다.  

---

### **QLoRA 주요 개념**  
1. **4비트 양자화:**  
   모델 가중치(Weights)를 4비트로 표현해 **메모리 사용량을 크게 줄입니다**.  

2. **동결된 파라미터:**  
   **기존 모델의 파라미터는 학습하지 않으며**, 대신 새로운 **LoRA 어댑터**에만 학습을 진행합니다.

3. **LoRA 어댑터:**  
   LoRA는 모델의 특정 부분에 **추가되는 가벼운 레이어**입니다. 전체 파라미터를 업데이트하지 않고 **LoRA 어댑터만 학습**하여 성능과 효율을 유지합니다.

4. **메모리 효율성:**  
   4비트 양자화와 LoRA의 조합은 매우 큰 모델도 **VRAM 16GB 수준의 GPU**에서 다룰 수 있도록 해줍니다.

---

### 2. **코드 분석**

```python
import torch
from transformers import BitsAndBytesConfig

# float16 데이터 타입 설정 (효율적이고 빠른 연산을 위해 사용)
compute

_dtype = getattr(torch, "float16")

# BitsAndBytes 라이브러리를 사용한 양자화 설정
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4비트 양자화를 사용하여 메모리 효율성 극대화
    bnb_4bit_quant_type="nf4",  # NF4 양자화 유형 선택 (정확도 향상)
    bnb_4bit_compute_dtype=compute_dtype,  # 연산에 float16 사용
    bnb_4bit_use_double_quant=False  # 이중 양자화 비활성화 (단순성과 속도 유지)
)
```

---

### **코드 구성 요소 설명**

1. **`load_in_4bit=True`**  
   모델 가중치를 **4비트로 로드**하여 메모리 사용량을 줄입니다.  

2. **`bnb_4bit_quant_type="nf4"`**  
   **NF4**는 일반적인 4비트보다 더 정확한 양자화 형식입니다. 가중치의 분포를 잘 반영하도록 설계되어 모델 성능이 유지됩니다.  

3. **`bnb_4bit_compute_dtype=compute_dtype`**  
   **float16** 데이터 타입을 사용해 연산 효율을 극대화합니다. 대부분의 GPU에서 **속도와 메모리 사용 간 균형**을 맞추기에 적합합니다.  

4. **`bnb_4bit_use_double_quant=False`**  
   이중 양자화(Double Quantization)를 비활성화하여 **복잡도를 줄이고** 더 빠르게 연산합니다.

---

### 3. **QLoRA 파인튜닝 전체 코드 예시**

아래는 Hugging Face `transformers` 라이브러리를 사용해 QLoRA를 구현하는 전체 코드입니다.

```python
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, Trainer, TrainingArguments

# 1. 양자화 설정
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# 2. 사전 학습된 모델 로드 (4비트 양자화 적용)
model_name = "bigscience/bloom-560m"  # 예시 모델
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,  # 양자화 설정 적용
    device_map="auto"  # 사용 가능한 장치(GPU)로 자동 매핑
)

# 3. 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # 패딩 토큰 처리

# 4. 학습 데이터 준비 (예시)
train_data = [
    {"input_text": "안녕, 기분이 어때?", "output_text": "좋아."},
    {"input_text": "너의 이름은 뭐야?", "output_text": "나는 QLoRA 봇이야."},
]

# 데이터 전처리 함수 정의
def preprocess_function(examples):
    inputs = tokenizer(examples["input_text"], truncation=True, padding="max_length", max_length=64)
    outputs = tokenizer(examples["output_text"], truncation=True, padding="max_length", max_length=64)
    inputs["labels"] = outputs["input_ids"]
    return inputs

train_dataset = [preprocess_function(data) for data in train_data]

# 5. 학습 설정
training_args = TrainingArguments(
    output_dir="./qlora_model",  # 모델이 저장될 경로
    per_device_train_batch_size=1,  # 배치 사이즈 설정
    gradient_accumulation_steps=16,  # 그래디언트 누적 단계
    evaluation_strategy="epoch",  # 에포크마다 평가
    save_strategy="epoch",  # 에포크마다 모델 저장
    logging_dir="./logs",  # 로그 디렉터리
    learning_rate=5e-4,  # 학습률 설정
    num_train_epochs=3,  # 학습 에포크 수
    weight_decay=0.01,  # 가중치 감소 설정
    push_to_hub=False,  # 허브 푸시 비활성화
    optim="adamw_torch"  # 최적화 기법 설정
)

# 6. Trainer 정의 (LoRA 어댑터만 학습)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)

# 7. 모델 학습
trainer.train()
```

---

### 4. **코드 작동 원리**

1. **4비트 양자화 모델 로드:**  
   모델을 4비트 형식으로 로드하여 **메모리 사용량을 크게 줄입니다**.

2. **기존 모델 파라미터 동결:**  
   **LoRA 어댑터만 학습**하며, 기존 모델의 가중치는 동결합니다.  

3. **데이터 준비 및 학습:**  
   작은 예제 데이터를 사용해 **파인튜닝**합니다. 실제 환경에서는 더 큰 데이터셋을 사용할 수 있습니다.  

4. **Trainer 사용:**  
   Hugging Face의 `Trainer`를 사용해 **자동으로 배치 처리 및 학습을 관리**합니다.

---

### 5. **QLoRA의 장점**

- **메모리 효율성:**  
  4비트 양자화로 **큰 모델도 적은 VRAM에서 실행**할 수 있습니다.

- **연산 효율성:**  
  **LoRA 어댑터만 학습**하므로 빠른 학습이 가능합니다.

- **확장성:**  
  QLoRA는 GPT-3, LLaMA와 같은 **대형 모델의 파인튜닝에도 적합**합니다.

---

이와 같은 QLoRA 방법을 사용하면 **제한된 자원**으로도 **대형 언어 모델을 효율적으로 파인튜닝**할 수 있습니다.

### **PEFT (Parameter-Efficient Fine-Tuning)**란?

**PEFT**는 모델 전체를 파인튜닝하지 않고, **일부 파라미터만 조정**하여 **효율적이고 경제적인 파인튜닝**을 가능하게 하는 기술입니다. 특히, **대규모 언어 모델(LLMs)**과 같은 거대한 모델에서는 모든 파라미터를 학습하기에는 **메모리와 연산 비용이 매우 큽니다**.  

PEFT는 이런 문제를 해결하기 위해 **LoRA**(Low-Rank Adaptation)와 같은 **어댑터 레이어**를 사용하여 특정 파라미터만 업데이트합니다. 이를 통해:
- **모델 학습 비용**을 줄이고,  
- **모델 성능 손실 없이 파인튜닝**이 가능합니다.  

---

## **LoRA (Low-Rank Adaptation)**와 PEFT  
LoRA는 **PEFT의 한 기법**으로, 원래의 모델 파라미터는 **고정(frozen)** 시켜두고, **작은 저랭크(rank) 어댑터 레이어**만 학습합니다. 이를 통해 작은 데이터셋으로도 효율적으로 **파인튜닝**이 가능합니다.

LoRA는 다음과 같은 원리로 동작합니다:
- 특정 레이어에 대한 **W = W₀ + ΔW** 형태로 업데이트합니다.  
- **W₀**는 원래 가중치, **ΔW**는 저랭크 어댑터를 통해 학습된 가중치입니다.
- ΔW는 두 저랭크 행렬(A와 B)로 나뉘며, **파라미터 수를 크게 줄여 메모리 효율성**을 높입니다.

---

## **PEFT를 사용한 LoRA 설정 코드 설명**

```python
from peft import LoraConfig

peft_params = LoraConfig(
    lora_alpha=16,         # LoRA의 스케일링 파라미터. 학습된 어댑터 가중치를 얼마나 크게 할지 조절.
    lora_dropout=0.1,       # 드롭아웃 확률. 학습 중 과적합(overfitting)을 방지하기 위해 일부 뉴런을 무작위로 비활성화.
    r=64,                   # 저랭크(rank) 설정. 작은 r 값을 사용해 학습되는 파라미터 수를 줄임.
    bias="none",            # 기존 모델의 바이어스 파라미터를 학습할지 여부 설정. 여기서는 사용 안 함.
    task_type="CAUSAL_LM",  # 파인튜닝할 작업 유형. 여기서는 'CAUSAL_LM' (Causal Language Model)로 설정.
)
```

---

### **PEFT 설정 옵션 설명**

1. **`lora_alpha=16`**  
   - **스케일링 파라미터**입니다. LoRA에서 학습된 어댑터의 가중치에 **알파(alpha)를 곱해 조정**합니다.  
   - 값이 클수록 학습된 어댑터의 영향이 커지고, 작을수록 적어집니다.

2. **`lora_dropout=0.1`**  
   - 학습 중 **과적합(overfitting)을 방지하기 위해** 드롭아웃을 적용합니다.  
   - 10% 확률로 일부 뉴런을 비활성화하여 **모델의 일반화 능력**을 향상시킵니다.

3. **`r=64`**  
   - **저랭크(rank)** 값입니다. 이 값은 학습할 저랭크 어댑터의 **크기를 결정**합니다.  
   - 낮은 `r` 값은 **학습해야 할 파라미터 수를 줄여** 메모리 사용을 줄이고 연산을 가볍게 만듭니다.

4. **`bias="none"`**  
   - 기존 모델의 **바이어스(bias) 파라미터**를 학습할지 여부를 지정합니다.  
   - `"none"`으로 설정하면 바이어스 파라미터는 **동결**됩니다.

5. **`task_type="CAUSAL_LM"`**  
   - 파인튜닝할 작업 유형입니다. 여기서는 **Causal Language Modeling**(자연어 생성 작업)을 수행하는 모델로 설정했습니다.  
   - 다른 작업 유형으로 **SequenceClassification** 등도 설정할 수 있습니다.

---

## **LoRA 적용을 위한 전체 코드 예시**

```python
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model

# 1. 모델과 토크나이저 로드
model_name = "gpt2"  # 예시로 GPT-2 모델 사용
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2. LoRA 설정
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

# 3. 모델에 LoRA 적용
model = get_peft_model(model, peft_params)

# 4. 학습 데이터 준비 (예시)
train_data = [
    {"input_text": "안녕하세요?", "output_text": "반갑습니다."},
    {"input_text": "오늘 날씨는 어때요?", "output_text": "맑고 따뜻합니다."}
]

def preprocess_function(examples):
    inputs = tokenizer(examples["input_text"], truncation=True, padding="max_length", max_length=64)
    outputs = tokenizer(examples["output_text"], truncation=True, padding="max_length", max_length=64)
    inputs["labels"] = outputs["input_ids"]
    return inputs

train_dataset = [preprocess_function(data) for data in train_data]

# 5. 학습 설정
training_args = TrainingArguments(
    output_dir="./peft_lora_model",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=5e-4,
    num_train_epochs=3,
    weight_decay=0.01
)

# 6. Trainer 정의
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer
)

# 7. 학습 시작
trainer.train()
```

---

### **PEFT 및 LoRA의 장점**

1. **메모리 절약**:  
   파라미터 전체를 학습하지 않으므로 **메모리 사용량이 크게 감소**합니다.  

2. **연산 효율성**:  
   저랭크 어댑터만 학습하므로 **연산 속도가 빠르고** 비용이 적게 듭니다.  

3. **확장성**:  
   대형 모델도 **작은 데이터셋**으로 파인튜닝할 수 있어 **다양한 응용**이 가능합니다.  

4. **유연성**:  
   다양한 작업 유형(언어 생성, 분류 등)에 적용할 수 있습니다.

---

이 코드 예시와 설명을 통해 **PEFT와 LoRA를 이용한 파인튜닝**의 개념과 사용법을 쉽게 이해할 수 있습니다. **대규모 모델을 효율적으로 학습**하고자 할 때 PEFT는 매우 유용한 기법입니다.

1. 필요 라이브러리 설치

- accelerate : 모델의 분산 학습 및 GPU/TPU 관리를 쉽게 도와줍니다.
- peft: **Parameter-Efficient Fine-Tuning(PEFT)**을 지원하는 라이브러리로 대형 언어 모델(LLM)의 일부 파라미터만 미세 조정(파인튜닝)하여 효율적인 파인튜닝을 가능하게 합니다.
  - **LoRA (Low-Rank Adaptation)**와 같은 기법 사용.
  - 메모리와 시간 소모를 줄이며 대형 모델을 파인튜닝.
  - Hugging Face 모델과 함께 쉽게 통합.
- bitsandbytes: 양자화(Quantization) 라이브러리로, 모델의 메모리 사용량을 줄이고 성능을 최적화합니다.
- Transformers: Hugging Face의 transformers 라이브러리는 다양한 **사전 학습된 언어 모델(LLM)**을 지원합니다.
- trl: TRL (Transformer Reinforcement Learning) 은 언어 모델을 학습할 때 강화학습(RL) 기법을 사용하기 위한 도구를 제공합니다.

In [ ]:
!pip install accelerate==0.26.1 peft==0.8.2 bitsandbytes==0.42.0 transformers==4.37.2 trl==0.7.10

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 4.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.1/108.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

hugging face 로그인

In [ ]:
import huggingface_hub
huggingface_hub.login()

2. 모델 설정 (본인 hugging dataset 폴더 참고 ★)

In [ ]:
# Hugging Face Basic Model 한국어 기본모델
base_model = "beomi/Llama-3-KoEn-8B-Instruct-preview"

# Custom Dataset ★ 본인이 hugging face 내 저장한 모델경로를 설정해야함 ★
bnv20_dataset = "bnv20/llama2dataset3"


3. 데이터 불러오기 (훈련)

In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


In [ ]:
base_model = "beomi/Llama-3-KoEn-8B-Instruct-preview"
bnv20_dataset = "bnv20/llama2dataset3"
dataset = load_dataset(bnv20_dataset, split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/271 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1288 [00:00<?, ? examples/s]

In [ ]:
# 데이터 확인
print( dataset[28] )

{'text': '<s>[INST] 철골구조가 뭐야? [/INST] 철골 구조는 수직의 강철 기둥과 수평의 형강으로 골격을 짜 건물의 하중을 떠받게 하고 내장재와 외장재 등은 커튼월 형식으로 장착하는 건축 기술입니다. </s>'}


4. 4비트 양자화 QLoRA 파인튜닝 (효율성) * 파라미터를 고정 시키고 추가데이터만 튜닝

In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True, #  모델의 가중치를 더 적은 비트로 표현하여 메모리 사용량을 크게 감소
    bnb_4bit_quant_type="nf4", # 양자화 유형을 설정
    bnb_4bit_compute_dtype=compute_dtype, # 용될 데이터 타입을 지정. 이 경우 torch.float16이 사용
    bnb_4bit_use_double_quant=False, # 양자화 과정에서 이중 양자화(double quantization)를 사용하지 않겠다는 의미
)

5. 라마모델 불러오기

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config, # 모델을 로드할 때 양자화 설정을 제공
    device_map={"": 0} # 모델의 각 부분을 특정 디바이스(예: CPU 또는 GPU)에 할당. 모델의 모든 파트를 GPU 0에 할당
)
model.config.use_cache = False
model.config.pretraining_tp = 1 #  모델의 텐서 병렬화(tensor parallelism) 설정을 정의합니다. 1로 설정하면 병렬 처리를 사용하지 않는 것을 의미

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/3.00G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

...

model-00004-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/1.29G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

6. 토크나이저 불러오기 (Hugginface에서 토크나이저를 로드하고 padding_side를 "right"로 설정하여 fp16과 관련된 문제해결)

In [ ]:
# trust_remote_code=True : 원격에서 로드된 토크나이저 코드가 사용자 정의 클래스나 함수를 포함할 수 있는 경우, 이를 실행하는 것을 허용
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token # 토크나이저의 패딩 토큰을 끝 토큰(EOS token, End of Sentence token)으로 설정
tokenizer.padding_side = "right" # 시퀀스를 패딩할 때 패딩을 오른쪽(시퀀스의 끝)에 적용하도록 지정

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### 7. PEFT 파라미터 (Parameter-Efficient Fine-Tuning (PEFT)은 모델 파라미터의 작은 하위 집합만 업데이트)

https://huggingface.co/docs/peft/conceptual_guides/lora

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

8. Training parameters

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
import torch
import gc

# 사용하지 않는 변수 삭제
# del 변수명
gc.collect()  # Python의 가비지 컬렉터를 사용하여 메모리 정리
torch.cuda.empty_cache()  # PyTorch가 GPU 메모리 캐시를 비움



9. model 파인튜닝

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1288 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Step,Training Loss
25,2.013800
50,1.678300
75,1.723100
100,1.512300
125,1.611100
150,1.302300
175,1.584300
200,1.478200
225,1.472000
250,1.511000


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=3864, training_loss=0.8805444715679556, metrics={'train_runtime': 1670.2284, 'train_samples_per_second': 2.313, 'train_steps_per_second': 2.313, 'total_flos': 1.535639821111296e+16, 'train_loss': 0.8805444715679556, 'epoch': 3.0})

## 평가

In [ ]:
from tensorboard import notebook
log_dir = "results/runs"
notebook.start("--logdir {} --port 4000".format(log_dir))

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "면진구조가 뭐야?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 면진구조가 뭐야? [/INST] 면진구조란 지반에서 오는 진동을 흡수할 수 있는 재료를 이용하여 지반과 건물을 분리시키는 진동 격리장치를 말합니다. 면진구조는 지반에서 오는 진동을 흡수할 수 있는 재료를 이용하여 지반과 건물을 분리시키는 진동 격리장치를 말합니다. 면진구조는 지반에서 오는 진동을 흡수할 수 있는 재료를 이용하여 지반과 건물을 분리시키는 진동 격리장치를 말합니다. 면진구조는 지반에서 오는 진동을 흡수할 수 있는 재료를 이용하여 지반과 건물을 분리시키는 진동 격리장치를 말합니다. 면진구조는 지반에서 오는 진동을


In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "철골구조가 뭐야?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 철골구조가 뭐야? [/INST] 철골구조는 강도와 강성이 우수한 강판과 형강, 평강 등을 용접하여 건축물의 외장 또는 내장 구조로 하는 것으로 건축물의 하중을 부담하는 주요한 구조 요소 중 하나입니다. 철골구조는 일반적으로 형강구조와 강판구조로 나뉩니다. 형강구조는 H형강, C형강, L형강, T형강 등과 같은 형강을 사용하여 수평, 수직으로 조립하여 건축물의 하중을 부담하는 구조로 강도와 강성이 우수하여 고층건물, 장대교량, 대형선박 등에 사용됩니다. 강판구조는 일반적으로 두께 1mm 이상의 강판을 용접하여 조립하는 구조로 형강구조와


In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "철골구조가 뭐야?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    temperature=0.7,  # 무작위성 증가
    top_k=50,         # 상위 50개 후보에서 샘플링
    top_p=0.9,        # 누적 확률이 0.9 이하인 후보에서만 샘플링
    max_length=200
)
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<s>[INST] 철골구조가 뭐야? [/INST] 철골구조는 강도와 강성이 좋은 강판과 형강, 평강 등을 용접하여 건축물의 뼈대로 하는 구조로 건물의 높이가 높아질수록 경제적이어서 현대 건축에 많이 쓰입니다. 장점으로는 시공의 용이성, 강도의 조절이 용이하다는 점이 있고, 단점으로는 화재 시 철골이 녹아 건물의 붕괴로 이어질 수 있다는 점, 지진에 취약하다는 점, 콘크리트 구조에 비해 비싸다는 점이 있습니다. </INST> 철골구조는 강도와 강성이 좋은 강판과 형강, 평강 등을 용접하여 건축물의 뼈대로 하는 구조로 건축물의 높이가 높아질수록 경제적이어서 현대 건축


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi


Thu Oct 17 14:52:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              53W / 400W |  14467MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
model.save_pretrained("/content/drive/MyDrive/kdt_240424/m9_llm/llama_finetuned")
tokenizer.save_pretrained("/content/drive/MyDrive/kdt_240424/m9_llm/llama_finetuned")

('/content/drive/MyDrive/kdt_240424/m9_llm/llama_finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/kdt_240424/m9_llm/llama_finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/kdt_240424/m9_llm/llama_finetuned/tokenizer.json')

In [ ]:
base_model = "beomi/Llama-3-KoEn-8B-Instruct-preview"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# 1. 로컬에 저장된 토크나이저와 기본 모델 로드
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/kdt_240424/m9_llm/llama_finetuned")
base_model = AutoModelForCausalLM.from_pretrained("beomi/Llama-3-KoEn-8B-Instruct-preview")

# 2. 로컬에 저장된 LoRA 모델 로드
model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/kdt_240424/m9_llm/llama_finetuned")


In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "면진구조가 뭐야?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
# Fine-tuned model
new_model = "Llama-3-KoEn-8B-bnv20-kor_v1"

In [ ]:
tokenizer.push_to_hub("bnv20/Llama-3-KoEn-8B-bnv20-kor_v1")

CommitInfo(commit_url='https://huggingface.co/bnv20/Llama-3-KoEn-8B-bnv20-kor_v1/commit/ebe78aa921b0cba41d3ede2e96ba654cae06a4a1', commit_message='Upload tokenizer', commit_description='', oid='ebe78aa921b0cba41d3ede2e96ba654cae06a4a1', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model.push_to_hub("bnv20/Llama-3-KoEn-8B-bnv20-kor_v1")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.24G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/bnv20/Llama-3-KoEn-8B-bnv20-kor_v1/commit/f765b657f47c6b2c1723c1463a3f5f65d02d1bad', commit_message='Upload LlamaForCausalLM', commit_description='', oid='f765b657f47c6b2c1723c1463a3f5f65d02d1bad', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("bnv20/Llama-3-KoEn-8B-bnv20-kor_v1")
model = AutoModelForCausalLM.from_pretrained(
    "bnv20/Llama-3-KoEn-8B-bnv20-kor_v1",
    device_map="auto"  # 자동으로 GPU/CPU에 매핑
)